# Extracting musiXmatch track IDs for Million Song Dataset from [SQLite DB](http://millionsongdataset.com/sites/default/files/AdditionalFiles/mxm_dataset.db) of the [musiXmatch dataset](http://millionsongdataset.com/musixmatch/)

This notebook documents how the musiXmatch track IDs we needed for our custom lyric dataset building on the [musiXmatch dataset](http://millionsongdataset.com/musixmatch/) were obtained from the [SQLite DB](http://millionsongdataset.com/sites/default/files/AdditionalFiles/mxm_dataset.db) of the dataset.

If you wish to reproduce the code here, download the file and put it into the `data` directory.

As the authors of the dataset mention on the dataset website,this is a subset of the [full mapping of MSD IDs to musiXmatch IDS](http://millionsongdataset.com/sites/default/files/AdditionalFiles/mxm_779k_matches.txt.zip). It contains 237,662 tracks (with lyrics in bag-of-words format) compared to the original 1 million tracks from the MSD dataset. Several tracks had to be removed for *"several reasons, including"*:
> * diverse restrictions, including copyrights
> * instrumental tracks
> * the numerous MSD duplicates were skipped as much as possible

In [1]:
import numpy as np
import pandas as pd
import sqlite3
import os
from io import BytesIO
from zipfile import ZipFile
import requests

## Read lyrics table

In [2]:
with sqlite3.connect("data/mxm_dataset.db") as con:
  tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", con)
  lyrics = pd.read_sql_query("SELECT * FROM lyrics;", con)


In [3]:
tables

,name
0,words
1,lyrics


In total, there's two tables, but only `lyrics` is of interest to us:

In [4]:
lyrics

,track_id,mxm_tid,word,count,is_test
0,TRAAAAV128F421A322,4623710,i,6,0
1,TRAAAAV128F421A322,4623710,the,4,0
2,TRAAAAV128F421A322,4623710,you,2,0
3,TRAAAAV128F421A322,4623710,to,2,0
4,TRAAAAV128F421A322,4623710,and,5,0
...,...,...,...,...,...
19045327,TRZZZZD128F4236844,2466899,easili,1,1
19045328,TRZZZZD128F4236844,2466899,disast,1,1
19045329,TRZZZZD128F4236844,2466899,frown,1,1
19045330,TRZZZZD128F4236844,2466899,teas,1,1


It contains bag-of-words-lyrics for all the songs in the dataset. However, we're only interested in the musiXmatch track IDs, as we want to obtain the original lyrics with the musiXmatch API for this project.

In [5]:
lyrics.mxm_tid.nunique()

237662

We can see that the dataset contains the number of `track_id`s described on the website.

## Extract relevant data

`track_id`,  `mxm_id` and `is_test` are the only relevant columns for our purposes. For convenience, we also rename `track_id` to `msd_tid`.

In [18]:
relevant = lyrics[["track_id", "mxm_tid", "is_test"]].drop_duplicates().rename(columns={"track_id":"msd_tid"})
relevant

,msd_tid,mxm_tid,is_test
0,TRAAAAV128F421A322,4623710,0
68,TRAAABD128F429CF47,6477168,0
142,TRAAAED128E0783FAB,2516445,0
262,TRAAAEF128F4273421,3759847,0
321,TRAAAEW128F42930C0,3783760,0
...,...,...,...
19044975,TRZZXFY128F9342D0E,1265451,1
19045043,TRZZXOQ128F932A083,4292070,1
19045092,TRZZXVN128F93285B4,7528751,1
19045190,TRZZYLF128F9316CAB,3748433,1


This dataframe contains the expected number of values.

## Check for duplicates

Let's check the obtained tracks against the official [list of duplicates](http://millionsongdataset.com/sites/default/files/AdditionalFiles/msd_duplicates.txt).

In [7]:
def get_remote_zip(url):
  resp = requests.get(url)
  return ZipFile(BytesIO(resp.content))

In [8]:
def get_remote_textfile(url):
  resp = requests.get(url)
  return resp.text

In [9]:
data_dir = "data"
if not os.path.exists(data_dir):
  os.mkdir(data_dir)

In [10]:
duplicate_file_path = os.path.join(data_dir, "msd_duplicates.txt")
if not os.path.exists(duplicate_file_path):
  duplicate_file_content = get_remote_textfile("http://millionsongdataset.com/sites/default/files/AdditionalFiles/msd_duplicates.txt")
  with open(duplicate_file_path, "w") as f:
    f.write(duplicate_file_content)

We have successfully loaded the list of duplicates!

It has the following structure:
```
% ARTIST - TITLE
DUPLICATE_ID1
...
DUPLICATE_ID_N
% NEXT_ARTIST - NEXT_TITLE
...
```

So, we have some processing to do. We assume that if we have a single match in terms of musiXmatch ID, we can remove all the remaining matches from our dataset.

In [11]:
import re

with open(duplicate_file_path) as f:
  # remove comments (lines starting with "#")
  duplicates_data = "".join([line for line in f.readlines() if not line.startswith("#")])
  # we could extract the artist and track names for the duplicates from the file, like this:
  # artist_and_track_name_strs = re.findall(r"\%[0-9]*\s(.*)\n", duplicates_data)
  # splits = [str.split(" - ") for str in artist_and_track_name_strs]

  # But actually there's a few instances where " - " is either part of the song title or the artist's name
  # there's no clear rule for splitting that without introducing error
  # so, rather than picking the names from here, we will just use the lists of duplicates

  duplicated_tracks = [ids_str.split("\n")[:-1] for ids_str in re.split(r"\%[0-9]*\s.*\n", duplicates_data)[1:]]

How many songs are actually duplicated?

In [12]:
len(duplicated_tracks)

53471

This perfectly coincides with the content of the file, it also lists 53471 combinations of artist and track title!

The authors of the duplicate list also state that in total 131661 "song objects" are duplicates of another one. Let's make sure we extracted the same information:

In [13]:
sum([len(track_ids) for track_ids in duplicated_tracks])

131661

Looks good! 

Now we can check the extracted lyric data against the duplicates list.

For this purpose, we first create a DataFrame with two columns: a "duplicate ID" that uniquely identifies one of the 54771 duplicated tracks and the respective track ID from the MSD dataset.

In [14]:
dups_and_tracks = [[(i, track_id) for track_id in track_ids] for (i, track_ids) in enumerate(duplicated_tracks)]
dups_and_tracks_flat = [item for dup_and_tracks in dups_and_tracks for item in dup_and_tracks]
dup_mapping = pd.DataFrame(dups_and_tracks_flat, columns=["duplicate_id", "msd_tid"])
dup_mapping

,duplicate_id,msd_tid
0,0,TRFCVSW12903D0A298
1,0,TRCWFEM128F9320F94
2,0,TRKYJRK12903CE6493
3,0,TRWTOBV128F9300F8A
4,1,TRWFIGX128F42920CA
...,...,...
131656,53468,TRVTTQH12903C9B37B
131657,53469,TRDNEDV128F92FFE25
131658,53469,TRUYBTI128F422D6CC
131659,53470,TRXVMUN128E0784025


In [24]:
duplicate_counts = pd.merge(relevant, dup_mapping, on="msd_tid").groupby("duplicate_id").size()
duplicate_counts[duplicate_counts > 1]

Series([], dtype: int64)

The fact that the code above returned an empty Series confirms that there indeed are no duplicates left. Nice!

## Write data to CSV

In [25]:
relevant.to_csv("data/msd_to_mxm.csv", index=False)